In [447]:
print("womp womp")

womp womp
time: 0 ns (started: 2023-12-09 19:57:41 -05:00)


In [448]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 31 ms (started: 2023-12-09 19:57:41 -05:00)


In [449]:
file_path = r"C:\Users\aaron\Documents\Python Code\Datasets\Heart Attack Risk Predicition\heart_attack_prediction_dataset.csv"

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,67,Male,208,72,0,0,1,0,0,4.168189,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,21,Male,389,98,1,1,1,1,1,1.813242,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,21,Female,324,72,1,0,0,0,0,2.078353,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,84,Male,383,73,1,1,1,0,1,9.828130,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,66,Male,318,93,1,1,1,1,0,5.804299,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


time: 32 ms (started: 2023-12-09 19:57:41 -05:00)


In [450]:
#Defining mapping function to map multiple inputs of the dataset
def gender_map(x):
    return x.map({'M': 1, 'F': 0, 'Male' : 1, "Female" : 0, '1' : 1, '0' : 0})

def hemisphere_map(x):
    return x.map({'Southern Hemisphere' : -1, 'Northern Hemisphere' : 1})

def countries_map(x):
    return x.map({'Argentina' : -1, 'Brazil' : -1, 'China' : -1, 'Colombia' : -1,
    'India' : -1, 'Nigeria' : -1, 'South Africa' : -1, 'South Korea' : -1, 'Thailand' : -1,
    'Vietnam' : -1, 'Australia' : 1, 'Canada' : 1, 'France' : 1, 'Germany' : 1,
    'Italy' : 1, 'Japan' : 1, 'New Zealand' : 1, 'Spain' : 1, 'United Kingdom' : 1,
    'United States' : 1})

def diet_map(x):
    return x.map({'Unhealthy' : -1, 'Average' : 0, 'Healthy' : 1})

def continent_map(x):
    return x.map({'Asia' : 0, 'Africa' : 1, 'Europe' : 2, 'North America' : 3,
    'South America' : 4, 'Australia' : 5})

time: 0 ns (started: 2023-12-09 19:57:41 -05:00)


In [451]:
heart_attack['Sex'] = heart_attack[['Sex']].apply(gender_map)
heart_attack['Hemisphere'] = heart_attack[['Hemisphere']].apply(hemisphere_map)
heart_attack['Country'] = heart_attack[['Country']].apply(countries_map)
heart_attack['Diet'] = heart_attack[['Diet']].apply(diet_map)
heart_attack['Continent'] = heart_attack[['Continent']].apply(continent_map)

unique_sex = heart_attack['Sex'].unique()
print(unique_sex)
unique = heart_attack['Hemisphere'].unique()
print(unique)
unique_country = heart_attack['Country'].unique()
print(unique_country)
unique_diet = heart_attack['Diet'].unique()
print(unique_diet)
unique_continent = heart_attack['Continent'].unique()
print(unique_continent)

heart_attack.head()

[1 0]
[-1  1]
[-1  1]
[ 0 -1  1]
[4 3 2 0 1 5]


,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,67,1,208,72,0,0,1,0,0,4.168189,...,6.615001,261404,31.251233,286,0,6,-1,4,-1,0
1,21,1,389,98,1,1,1,1,1,1.813242,...,4.963459,285768,27.194973,235,1,7,1,3,1,0
2,21,0,324,72,1,0,0,0,0,2.078353,...,9.463426,235282,28.176571,587,4,4,1,2,1,0
3,84,1,383,73,1,1,1,0,1,9.828130,...,7.648981,125640,36.464704,378,3,4,1,3,1,0
4,66,1,318,93,1,1,1,1,0,5.804299,...,1.514821,160555,21.809144,231,1,5,-1,0,1,0


time: 15 ms (started: 2023-12-09 19:57:41 -05:00)


In [452]:
mode_columns = ['Stress Level','Medication Use', 'Previous Heart Problems', 'Diet', 'Physical Activity Days Per Week', 'Alcohol Consumption', 'Obesity', 'Smoking', 
                'Family History', 'Diabetes', 'Hemisphere', 'Continent', 'Country']

mean_columns = ['Exercise Hours Per Week', 'Sleep Hours Per Day', 'BMI', 'Sedentary Hours Per Day']

median_columns = ['Heart Rate', 'Income', 'Triglycerides']

# Fill NaN values with the mode of the column for mode_columns
for col in mode_columns:
    heart_attack[col].fillna(heart_attack[col].mode()[0], inplace=True)

# Fill NaN values with the median of the column for median_columns
for col in median_columns:
    heart_attack[col].fillna(heart_attack[col].median(), inplace=True)

# Fill NaN values with the mean of the column for mean_columns
for col in mean_columns:
    heart_attack[col].fillna(heart_attack[col].mean(), inplace=True)

# heart_attack.fillna(-1e10, inplace=True)


unique_sex = heart_attack['Sex'].unique()
print(unique_sex)
unique = heart_attack['Hemisphere'].unique()
print(unique)
unique_country = heart_attack['Country'].unique()
print(unique_country)
unique_diet = heart_attack['Diet'].unique()
print(unique_diet)
unique_continent = heart_attack['Continent'].unique()
print(unique_continent)


# Check for NaN values in columns after fillna operations
columns_with_nulls = heart_attack.columns[heart_attack.isnull().any()].tolist()
print("Columns with NaN values after fillna operations:")
print(columns_with_nulls)

heart_attack.head()

[1 0]
[-1  1]
[-1  1]
[ 0 -1  1]
[4 3 2 0 1 5]
Columns with NaN values after fillna operations:
[]


,Age,Sex,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,67,1,208,72,0,0,1,0,0,4.168189,...,6.615001,261404,31.251233,286,0,6,-1,4,-1,0
1,21,1,389,98,1,1,1,1,1,1.813242,...,4.963459,285768,27.194973,235,1,7,1,3,1,0
2,21,0,324,72,1,0,0,0,0,2.078353,...,9.463426,235282,28.176571,587,4,4,1,2,1,0
3,84,1,383,73,1,1,1,0,1,9.828130,...,7.648981,125640,36.464704,378,3,4,1,3,1,0
4,66,1,318,93,1,1,1,1,0,5.804299,...,1.514821,160555,21.809144,231,1,5,-1,0,1,0


time: 16 ms (started: 2023-12-09 19:57:41 -05:00)


In [453]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
    print("GPU is available.")
else:
    device = torch.device("cpu")   # Use CPU
    print("GPU is not available, using CPU.")

GPU is not available, using CPU.
time: 0 ns (started: 2023-12-09 19:57:41 -05:00)


In [475]:
#Splitting the Variables into X and Y
X = heart_attack.iloc[:, 0:22].values
Y = heart_attack.iloc[:, 23].values

# Split our Data set into Training Data and val Data.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)

Y_train_t = torch.tensor(Y_train, dtype=torch.float)
Y_val_t = torch.tensor(Y_val, dtype=torch.float)


print(X_train_t.dtype)
print(X_val_t.dtype)
print(Y_train_t.dtype)
print(Y_val_t.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
torch.Size([7010, 1])
time: 16 ms (started: 2023-12-09 20:02:56 -05:00)


In [455]:
# Defining PCA function
def PCA_function(X_train, X_val, K):
  pca = PCA(n_components=K)
  X_train_PCA = pca.fit_transform(X_train)
  X_val_PCA = pca.fit_transform(X_val)
  return X_train_PCA, X_val_PCA

time: 0 ns (started: 2023-12-09 19:57:41 -05:00)


In [456]:
(X_train_PCA, X_val_PCA) = PCA_function(X_train, X_val, 3)

print("The shape of X_train after PCA is", X_train_PCA.shape)
print("The shape of X_val after PCA is", X_val_PCA.shape)

# Use standard scaling from Sklearn to scale data between - and  for better accuracy.
scale = StandardScaler()
X_train_PCA = scale.fit_transform(X_train_PCA)
X_val_PCA = scale.fit_transform(X_val_PCA)

X_train_PCA_t = torch.tensor(X_train_PCA, dtype=torch.float)
X_val_PCA_t = torch.tensor(X_val_PCA, dtype=torch.float)

print("\nThe shape of the X_train tensor is", X_train_PCA_t.shape)
print("The shape of the X_val tensor is", X_val_PCA_t.shape)

The shape of X_train after PCA is (7010, 3)
The shape of X_val after PCA is (1753, 3)

The shape of the X_train tensor is torch.Size([7010, 3])
The shape of the X_val tensor is torch.Size([1753, 3])
time: 16 ms (started: 2023-12-09 19:57:41 -05:00)


In [457]:
def model_linear(t_in, w1, w2, w3, b):
  t_p = ((w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b)
  return t_p

def loss_fn(t_p, t_gnd):
  squared_diffs = (t_p - t_gnd)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, val_t_in,
                    train_t_out, val_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model_linear(train_t_in, *params)
    train_loss = loss_fn(train_t_p, train_t_out)

    val_t_p = model_linear(val_t_in, *params)
    val_loss = loss_fn(val_t_p, val_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {val_loss.item():.4f}")

  return params

time: 0 ns (started: 2023-12-09 19:57:41 -05:00)


In [458]:
params = torch.tensor([1.0, 1.0, 1.0, 0.0], requires_grad=True)

learning_rate_1 = 1e-3
optimizer_1 = optim.SGD([params], lr=learning_rate_1)

print('Loss every 500 epochs using learning rate of 1e-3\n')
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params,
    train_t_in = X_train_PCA_t,
    val_t_in = X_val_PCA_t,
    train_t_out = Y_train_t,
    val_t_out = Y_val_t)

Loss every 500 epochs using learning rate of 1e-3

Epoch 1, 	Training loss 10.1827, 	Validation loss 9.9481
Epoch 2, 	Training loss 10.0053, 	Validation loss 9.7740
Epoch 3, 	Training loss 9.8311, 	Validation loss 9.6031


Epoch 500, 	Training loss 0.3781, 	Validation loss 0.3833
Epoch 1000, 	Training loss 0.3097, 	Validation loss 0.3144
Epoch 1500, 	Training loss 0.2760, 	Validation loss 0.2797
Epoch 2000, 	Training loss 0.2574, 	Validation loss 0.2604
Epoch 2500, 	Training loss 0.2469, 	Validation loss 0.2494
Epoch 3000, 	Training loss 0.2407, 	Validation loss 0.2430
Epoch 3500, 	Training loss 0.2370, 	Validation loss 0.2391
Epoch 4000, 	Training loss 0.2347, 	Validation loss 0.2367
Epoch 4500, 	Training loss 0.2332, 	Validation loss 0.2351
Epoch 5000, 	Training loss 0.2322, 	Validation loss 0.2341


tensor([ 0.1113,  0.0252, -0.0552,  0.3235], requires_grad=True)

time: 2.53 s (started: 2023-12-09 19:57:41 -05:00)


In [478]:
model_output = model_linear(X_train_PCA_t, *params)

# print(model_output)
# print(model_output.shape)

model_output_np = model_output.detach().numpy() if isinstance(model_output, torch.Tensor) else model_output

# For example, using argmax to get the class with the highest probability
predicted_labels = np.argmax(model_output_np, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_train, predicted_labels)

print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.3357
time: 0 ns (started: 2023-12-09 20:05:36 -05:00)


In [459]:
# PCA_prediction = model_linear(X_train_PCA_t, *params)
# print(PCA_prediction.shape)
# print(PCA_prediction[:5])

# # Calculate accuracy, precision, recall, and F1 score of training set
# train_accuracy = accuracy_score(Y_train_t.detach().numpy(), PCA_prediction.detach().numpy())

# train_precision = precision_score(Y_train_t.detach().numpy(), PCA_prediction.detach().numpy(), average='binary')

# train_recall = recall_score(Y_train_t.detach().numpy(), PCA_prediction.detach().numpy(), average='binary')

# train_f1 = f1_score(Y_train_t.detach().numpy(), PCA_prediction.detach().numpy(), average='binary')


# PCA_prediction_val = model_linear(X_val_PCA_t, *params)
# print(PCA_prediction_val.shape)

# # Calculate accuracy, precision, recall, and F1 score of validation set
# val_accuracy = accuracy_score(Y_val_t.detach().numpy(), PCA_prediction_val.detach().numpy())

# val_precision = precision_score(Y_val_t.detach().numpy(), PCA_prediction_val.detach().numpy(), average='binary')

# val_recall = recall_score(Y_val_t.detach().numpy(), PCA_prediction_val.detach().numpy(), average='binary')

# val_f1 = f1_score(Y_val_t.detach().numpy(), PCA_prediction_val.detach().numpy(), average='binary')

time: 0 ns (started: 2023-12-09 19:57:44 -05:00)


In [460]:
# # Convert multilabel-indicator to single-label format
# Y_train_single = np.argmax(Y_train_t.detach().numpy(), axis=1)
# Y_val_single = np.argmax(Y_val_t.detach().numpy(), axis=1)

# # Calculate accuracy, precision, and recall for training set
# train_accuracy = accuracy_score(Y_train_single, train_pred_labels)
# train_precision = precision_score(Y_train_single, train_pred_labels)
# train_recall = recall_score(Y_train_single, train_pred_labels)

# # Calculate accuracy, precision, and recall for validation set
# val_accuracy = accuracy_score(Y_val_single, val_pred_labels)
# val_precision = precision_score(Y_val_single, val_pred_labels)
# val_recall = recall_score(Y_val_single, val_pred_labels)

# # Print the calculated metrics
# print(f"Training Accuracy: {train_accuracy:.4f}")
# print(f"Training Precision: {train_precision:.4f}")
# print(f"Training Recall: {train_recall:.4f}")

# print(f"Validation Accuracy: {val_accuracy:.4f}")
# print(f"Validation Precision: {val_precision:.4f}")
# print(f"Validation Recall: {val_recall:.4f}")

time: 15 ms (started: 2023-12-09 19:57:44 -05:00)


In [461]:
#Sequential Model

model_seq = nn.Sequential(
    nn.Linear(3, 1024),
    nn.Tanh(),
    nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5 (adjust as needed)
    nn.Linear(1024, 2048),
    nn.Tanh(),
    nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5 (adjust as needed)
    nn.Linear(2048, 512),
    nn.Tanh(),
    nn.Linear(512, 1)
            )
model_seq

Sequential(
  (0): Linear(in_features=3, out_features=1024, bias=True)
  (1): Tanh()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=1024, out_features=2048, bias=True)
  (4): Tanh()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=2048, out_features=512, bias=True)
  (7): Tanh()
  (8): Linear(in_features=512, out_features=1, bias=True)
)

time: 16 ms (started: 2023-12-09 19:57:44 -05:00)


In [462]:
sum([p.numel() for p in model_seq.parameters()])

3152897

time: 0 ns (started: 2023-12-09 19:57:44 -05:00)


In [463]:
#Let's define our training loop that will be used

def training_loop_seq(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()


        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

time: 15 ms (started: 2023-12-09 19:57:44 -05:00)


In [464]:
#Let's try doing some training

optimizer_1 = optim.Adam(model_seq.parameters(), lr=1e-4)

training_loop_seq(
    n_epochs = 151,
    optimizer = optimizer_1,
    model = model_seq,
    loss_fn = nn.MSELoss(),
    x_train = X_train_PCA_t,
    x_val = X_val_PCA_t,
    y_train = Y_train_t,
    y_val = Y_val_t
)

Epoch: 1, Training loss: 0.3930, Validation loss: 0.3967
Epoch: 10, Training loss: 0.2558, Validation loss: 0.2590
Epoch: 20, Training loss: 0.2485, Validation loss: 0.2503
Epoch: 30, Training loss: 0.2403, Validation loss: 0.2388
Epoch: 40, Training loss: 0.2364, Validation loss: 0.2363
Epoch: 50, Training loss: 0.2366, Validation loss: 0.2376
Epoch: 60, Training loss: 0.2342, Validation loss: 0.2390
Epoch: 70, Training loss: 0.2355, Validation loss: 0.2384
Epoch: 80, Training loss: 0.2349, Validation loss: 0.2346
Epoch: 90, Training loss: 0.2334, Validation loss: 0.2390
Epoch: 100, Training loss: 0.2356, Validation loss: 0.2360
Epoch: 110, Training loss: 0.2334, Validation loss: 0.2374
Epoch: 120, Training loss: 0.2335, Validation loss: 0.2369
Epoch: 130, Training loss: 0.2331, Validation loss: 0.2339
Epoch: 140, Training loss: 0.2338, Validation loss: 0.2369
Epoch: 150, Training loss: 0.2312, Validation loss: 0.2354
time: 1min 38s (started: 2023-12-09 19:57:44 -05:00)


In [465]:
# Function to calculate accuracy
def calculate_accuracy(predictions, targets):
    return (predictions == targets).float().mean().item()

# Function to calculate recall
def calculate_recall(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_negatives = ((predictions == 0) & (targets == 1)).sum().item()
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Function to calculate precision
def calculate_precision(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_positive = ((predictions == 1) & (targets == 0)).sum().item()
    return true_positives / (true_positives + false_positive) if (true_positives + false_positive) > 0 else 0

time: 0 ns (started: 2023-12-09 19:59:23 -05:00)


In [466]:
# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_train_t)))

# Calculate accuracy for the training set
accuracy_train = calculate_accuracy(train_predictions, Y_train_t)
print("Training Accuracy: {:.4f}".format(accuracy_train))

# Calculate accuracy for the training set
recall_train = calculate_recall(train_predictions, Y_train_t)
print("Training Recall: {:.4f}".format(recall_train))

# Calculate accuracy for the training set
precision_train = calculate_precision(train_predictions, Y_train_t)
print("Training Precision: {:.4f}".format(precision_train))

# Calculate accuracy for the training set
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)
print("Training F1 Score: {:.4f}".format(F1_score_train))


# Calculate accuracy for the validation set
accuracy_val = calculate_accuracy(train_predictions, Y_val_t)
print("Validation Accuracy: {:.4f}".format(accuracy_val))

# Calculate accuracy for the validation set
recall_val = calculate_recall(train_predictions, Y_val_t)
print("Validation Recall: {:.4f}".format(recall_val))

# Calculate accuracy for the validation set
precision_val = calculate_precision(train_predictions, Y_val_t)
print("Validation Precision: {:.4f}".format(precision_val))

# Calculate accuracy for the validation set
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)
print("Validation F1 Score: {:.4f}".format(F1_score_val))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (7987x23 and 3x1024)

time: 62 ms (started: 2023-12-09 19:59:23 -05:00)
